In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [30]:
pv = pd.read_csv('./processed_data/pv/RISE_2021_PV_no_interpol.csv', index_col=0)
forecast_weather = pd.read_csv('./processed_data/weather/forecast_weather.csv')
observation_weather = pd.read_csv('./preprocessed_data/weather_data/weather_observation.csv', index_col=2, encoding='cp949').drop(['지점', '지점명'], axis=1)
observation_weather.columns = ['DS', 'SL', 'SR']  # (DS) Duration of Sunlight: 가조시간(hr) | (SL) SunLight 합계 일조시간(hr) | (SR) Solar Radiation 합계 일사량(MJ/m2)
load = pd.read_csv('./processed_data/week_2021_load.csv', index_col=0)
cal = pd.read_csv('./processed_data/2021_cal_flag.csv', index_col=0)
display(pv)
display(forecast_weather)
display(observation_weather)
display(load)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,...,21.8,11.0,15.1,0.5,0.0,0.0,0.0,0.0,0.0,0.0
210102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.3,...,37.7,32.3,21.5,5.0,0.0,0.0,0.0,0.0,0.0,0.0
210103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,9.9,...,29.1,8.5,3.9,0.7,0.0,0.0,0.0,0.0,0.0,0.0
210104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,10.3,...,41.7,10.2,5.8,0.9,0.0,0.0,0.0,0.0,0.0,0.0
210105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,7.4,...,23.9,9.8,7.5,0.9,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.3,...,21.6,4.7,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,6.6,...,7.3,5.7,1.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0
211229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,5.7,...,5.8,4.7,14.6,0.8,0.0,0.0,0.0,0.0,0.0,0.0
211230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,12.4,...,44.1,6.3,0.9,0.9,0.0,0.0,0.0,0.0,0.0,0.0


,TM_6,TM_9,TM_12,TM_15,TM_18,WS_6,WS_9,WS_12,WS_15,WS_18,...,SK_5,SK_18,PP_6,PP_9,PP_12,PP_15,PP_18,PR_9,PR_15,PR_21
0,1.0,-2.0,-4.0,-7.0,-6.0,4.1,1.6,2.3,2.4,3.0,...,3.0,4.0,20.0,30.0,20.0,20.0,30.0,0.0,0.0,0.0
1,2.0,-1.0,-4.0,-5.0,-6.0,3.4,1.5,1.0,1.3,1.5,...,3.0,3.0,20.0,20.0,20.0,20.0,20.0,0.0,0.0,0.0
2,4.0,1.0,-1.0,-1.0,0.0,0.4,0.9,1.6,2.3,0.5,...,3.0,4.0,0.0,0.0,10.0,20.0,30.0,0.0,0.0,1.0
3,-1.0,-2.0,-4.0,-5.0,-6.0,6.1,3.3,2.7,2.1,2.8,...,4.0,4.0,30.0,30.0,30.0,30.0,30.0,0.0,0.0,0.0
4,0.0,-2.0,-2.0,-2.0,-5.0,2.4,1.1,2.5,6.0,6.6,...,4.0,4.0,30.0,30.0,30.0,60.0,80.0,0.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,8.0,4.0,1.0,0.0,-1.0,2.6,1.3,1.1,1.4,1.2,...,4.0,4.0,30.0,30.0,30.0,30.0,30.0,0.0,0.0,0.0
361,8.0,5.0,3.0,1.0,0.0,4.7,2.0,3.2,2.8,3.0,...,4.0,4.0,30.0,30.0,30.0,30.0,30.0,0.0,0.0,0.0
362,4.0,2.0,0.0,-2.0,-4.0,6.5,3.1,3.3,3.7,4.2,...,4.0,4.0,60.0,30.0,30.0,60.0,60.0,0.0,0.0,0.0
363,0.0,-3.0,-5.0,-6.0,-6.0,4.7,2.2,2.5,2.4,2.5,...,3.0,3.0,20.0,20.0,0.0,20.0,20.0,0.0,0.0,0.0


,DS,SL,SR
일시,,,
2021-01-01,9.8,4.3,9.00
2021-01-02,9.9,8.6,12.54
2021-01-03,9.9,6.1,10.37
2021-01-04,9.9,5.8,10.07
2021-01-05,9.9,3.0,7.74
...,...,...,...
2021-12-27,9.8,6.5,11.35
2021-12-28,9.8,2.4,7.49
2021-12-29,9.8,3.6,7.77


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210104,879.6,906.7,902.6,866.7,821.9,869.8,853.7,893.9,1064.5,1110.0,...,1144.9,1109.3,1111.4,1101.6,1043.8,1016.0,975.6,979.7,982.6,933.4
210105,925.6,936.7,882.7,894.6,864.2,859.1,853.7,891.3,958.0,1092.0,...,1183.3,1183.1,1176.0,1150.8,1105.5,976.7,978.8,989.3,965.6,968.6
210106,941.6,938.5,908.3,922.8,889.1,909.2,890.2,958.8,1094.5,1179.1,...,1150.6,1128.4,1143.2,1150.4,1040.5,1043.8,1018.8,1017.6,978.0,958.3
210107,959.1,993.6,928.9,974.8,1028.6,985.0,971.7,1021.1,1077.4,1266.2,...,1351.0,1352.2,1358.4,1349.5,1279.1,1213.7,1264.2,1229.0,1223.8,1205.3
210108,1190.3,1182.8,1160.8,1122.6,1099.7,1116.8,1140.9,1152.3,1314.6,1387.3,...,1343.1,1316.1,1318.6,1346.2,1326.0,1207.8,1231.6,1258.3,1264.5,1202.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211221,827.8,788.2,780.5,805.4,746.8,787.1,822.2,810.5,951.3,1035.0,...,914.3,928.4,951.3,934.9,872.9,883.3,833.0,876.1,830.5,808.8
211227,862.1,868.6,836.0,849.5,886.9,861.9,888.2,896.7,1085.6,1188.9,...,1148.9,1148.0,1152.9,1139.0,1111.6,1011.1,1026.3,946.0,938.4,922.8
211228,885.6,890.9,859.7,875.7,868.3,853.8,850.5,868.5,959.1,1131.5,...,1141.7,1108.2,1072.2,1092.0,997.6,928.1,933.0,862.6,902.8,896.1
211229,847.5,877.3,824.7,813.5,836.8,774.4,809.9,853.0,977.0,1027.3,...,965.2,981.1,1013.5,957.5,893.3,871.3,902.9,878.8,910.0,869.2


In [31]:
# modify date(일시) string into integer to convert the value to tensor.
# set '일시' as index of observation_weather

observation_weather_len = len(observation_weather)
observation_weather_date_list = observation_weather.index
mdi_observation_weather_date_list = []

for i in range(observation_weather_len):
    date = observation_weather_date_list[i]
    date_time_obj = datetime.strptime(date, '%Y-%m-%d')
    mdi_date = (date_time_obj.year-2000)*10000 + date_time_obj.month*100 + date_time_obj.day
    mdi_observation_weather_date_list.append(mdi_date)
    
    
observation_weather.index = mdi_observation_weather_date_list
forecast_weather.index = mdi_observation_weather_date_list

comb = pd.concat([pv, observation_weather, forecast_weather], axis=1, join='outer')
# 50 features = 24 PV + 3 observation + 23 forecast
#  PV generation each hour
#  'DS', 'SL', 'SR'
#  'TM_6', 'TM_9', 'TM_12', 'TM_15', 'TM_18', 'WS_6', 'WS_9', 'WS_12', 'WS_15', 'WS_18',
#  'SK_6', 'SK_9', 'SK_12', 'SK_5', 'SK_18', 'PP_6', 'PP_9', 'PP_12', 'PP_15', 'PP_18',
#  'PR_9', 'PR_15', 'PR_21'

# check if comb has NaN values
np.isnan(comb.iloc[:,25:27]).any()

# find the index of NaN values
np.where(np.isnan(comb.iloc[:,25:27]))

# drop the NaN values
# comb = comb.drop([210618, 210930], axis=0)

(array([168, 168, 272, 272], dtype=int64), array([0, 1, 0, 1], dtype=int64))

In [32]:
comb.sort_index(inplace=True)

In [33]:
comb.iloc[355:360]

,0,1,2,3,4,5,6,7,8,9,...,SK_5,SK_18,PP_6,PP_9,PP_12,PP_15,PP_18,PR_9,PR_15,PR_21
211222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,1.0,0.0,20.0,20.0,30.0,0.0,0.0,0.0,0.0
211223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,4.0,30.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0
211224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,30.0,30.0,60.0,60.0,60.0,0.0,0.0,0.0
211225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,...,4.0,4.0,60.0,60.0,60.0,60.0,30.0,0.0,0.0,0.0
211226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,7.3,...,4.0,4.0,30.0,30.0,30.0,30.0,30.0,0.0,0.0,0.0


In [39]:
idx_intersection = list(set(comb.index) & set(load.index))
idx_intersection.sort()
# idx_intersection type: list

idx_len = len(idx_intersection)

cal_used = cal[['miss&week']]

In [47]:
cal_used.loc[cal_used['miss&week'] = 210101]

SyntaxError: invalid syntax (3789146808.py, line 1)

In [37]:
for idx, row in comb.iterrows():
    if cal_used[idx]

(210101, 0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.6
10        3.2
11        4.0
12        8.2
13       19.5
14       21.8
15       11.0
16       15.1
17        0.5
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
DS        9.8
SL        4.3
SR        9.0
TM_6      1.0
TM_9     -2.0
TM_12    -4.0
TM_15    -7.0
TM_18    -6.0
WS_6      4.1
WS_9      1.6
WS_12     2.3
WS_15     2.4
WS_18     3.0
SK_6      3.0
SK_9      4.0
SK_12     3.0
SK_5      3.0
SK_18     4.0
PP_6     20.0
PP_9     30.0
PP_12    20.0
PP_15    20.0
PP_18    30.0
PR_9      0.0
PR_15     0.0
PR_21     0.0
Name: 210101, dtype: float64)
(210102, 0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
5         0.00
6         0.00
7         0.00
8         0.00
9         4.30
10       12.20
11       23.30
12       35.40
13       38.00
14       37.70
15       32.30
16       21.

In [36]:
cal

,date_num,holi_flag,eve_holi_flag,week_flag,eve_week_flag,miss_flag_net,miss_flag_pv,miss_intersection,miss&week
210101,1,1,1,0,0,0,0,0,1
210102,2,1,1,0,0,0,0,0,1
210103,3,1,0,0,1,0,0,0,1
210104,4,0,0,1,1,0,0,0,0
210105,5,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
211227,361,0,0,1,1,0,0,0,0
211228,362,0,0,1,1,0,0,0,0
211229,363,0,0,1,1,0,0,0,0
211230,364,0,0,1,1,0,0,0,0


In [28]:
# # change the nan value to 0 in cal
# cal = cal.astype('int')
# cal.to_csv('./processed_data/2021_cal_flag.csv')